We start by importing necessary packages

In [198]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
import numpy as np
import seaborn as sns
import scipy as sc
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import r2_score
from xgboost import XGBRegressor

After importing the packages, we import the dataset.
We then clean the dataset by removing, from the copy that is now stored in the ram, the book titles that are not imported correctly or the data is wrongly transformed.
We also remove all those titles whos rating count is 0 as they don't have any rating that is the all neccessary response variable.

In [199]:
warnings.filterwarnings("ignore")
df = pd.read_csv(r"books.csv",sep=",", error_bad_lines=False, index_col="bookID")
df.columns = df.columns.str.replace(" ", "")
df = df[df.ratings_count != 0]
df = df[df['num_pages'].apply(lambda x: isinstance(x, (int, float)))]

b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


After having cleaned the dataset, we choose only to keep those variables that are not randomly assigned to the book. For example, the isbn number is not kept.

In [200]:
df = df.drop(columns=['isbn', 'isbn13'])

We then take on the remaining variables and and transform and/or combine them into new variables that we believe could enhance the model.

For example: Publishing date can potentially be quite pointless in itself, but if we make the published date relative to todays date by counting the number of days, we will know how long it is since the book was published.
By doing this, we can also find out the rating frequency by taking the number of ratings and dividing it by the number of days since published.

Some variables are also duplicated into a logarithmic version. This is done by the reasoning of for example if it turns out that an increase in the age of books generally corresponds with a better rating, then a book that is 1000 years old would be much better rated than any book published the last 50 years, which is a very large portion of the books one find on goodreads.com.

The first step in this is to convert 'publication_date' into a dataframe format.
We remove all titles that are not comatible with this conversion.

In [201]:
df["publication_date"] = pd.to_datetime(df["publication_date"], errors='coerce')
df = df.dropna();

We then create a categorical variable to find out if the language of the book is english or not. This is because one can assume there are many books written in different languages, but there are also many languages.
Creating a model based on each language could cause some very weak data (few datapoints) for some languages.

In [202]:
# Create Categorical variable. 1 if English, 0 if not English
df["is_eng"] = df["language_code"].str.contains('eng|en-').astype(int)
df.drop(columns=["language_code"]);

We also try to filter out audiobooks.

In [203]:
# Remove audiobooks
def is_audio(row):
    if row['num_pages'] < 30 or "audio" in row["publisher"]:
        return 1
    else:
        return 0

df["audio"] = df.apply(lambda row: is_audio(row), axis=1)
df = df[df.audio == 0]

We then start by creating the transformed variables.
We assume that the data was extracted at the end of December 2022.

In [204]:
specific_date_str = "2022-12-31" #yyyy-mm-dd
specific_date = pd.to_datetime(specific_date_str)
# Find time since the books were published
df["days_since_published"] = df["publication_date"].apply(lambda x: (specific_date - x).days)
df["years_since_published"] = df.days_since_published / 365.24
df["log_days_since_published"] = np.clip(np.log(df.days_since_published), 0, np.inf)

In [205]:
df["log_num_pages"] = np.log(df.num_pages)
df["log_r_count"] = np.log(df.ratings_count)
df["log_t_count"] = np.clip(np.log(df.text_reviews_count), 0, np.inf)

In [206]:
# Create variable that is rating/text_review per day/year/log_day (rating/review density)
df["r_count_per_day"] = df.ratings_count / df.days_since_published
df["t_count_per_day"] = df.text_reviews_count / df.days_since_published
df["r_count_per_year"] = df.ratings_count / df.years_since_published
df["t_count_per_year"] = df.text_reviews_count / df.years_since_published
df["r_count_per_log_day"] = df.ratings_count / df.log_days_since_published
df["t_count_per_log_day"] = df.text_reviews_count / df.log_days_since_published

In [207]:
df["log_r_count_per_day"] = df.log_r_count / df.days_since_published
df["log_t_count_per_day"] = df.log_t_count / df.days_since_published
df["log_r_count_per_year"] = df.log_r_count / df.years_since_published
df["log_t_count_per_year"] = df.log_t_count / df.years_since_published
df["log_r_count_per_log_day"] = df.log_r_count / df.log_days_since_published
df["log_t_count_per_log_day"] = df.log_t_count / df.log_days_since_published

In [208]:
df["ratings_count_per_num_pages"] = df.ratings_count / df.num_pages
df["text_reviews_count_per_num_pages"] = df.text_reviews_count / df.num_pages
df["ratings_count_per_log_num_pages"] = df.ratings_count / df.log_num_pages
df["text_reviews_count_per_log_num_pages"] = df.text_reviews_count / df.log_num_pages
df["log_r_count_per_num_pages"] = df.log_r_count / df.num_pages
df["log_t_count_per_num_pages"] = df.log_t_count / df.num_pages
df["log_r_count_per_log_num_pages"] = df.log_r_count / df.log_num_pages
df["log_t_count_per_log_num_pages"] = df.log_t_count / df.log_num_pages

As we of now have not applied any method for converting the string variables into numerical variables, we drop them from the dataset.

In [188]:
df.head(2)

,title,authors,average_rating,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,is_eng,...,log_r_count_per_log_day,log_t_count_per_log_day,ratings_count_per_num_pages,text_reviews_count_per_num_pages,ratings_count_per_log_num_pages,text_reviews_count_per_log_num_pages,log_r_count_per_num_pages,log_t_count_per_num_pages,log_r_count_per_log_num_pages,log_t_count_per_log_num_pages
bookID,,,,,,,,,,,,,,,,,,,,,
1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,1,...,1.674738,1.176513,3214.248466,42.317485,323406.726601,4257.841090,0.022324,0.015683,2.246187,1.577959
2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,1,...,1.655382,1.167273,2474.904598,33.587356,318116.149735,4317.209028,0.016761,0.011819,2.154460,1.519192


In [209]:
df = df.drop(columns=['title', 'authors', 'language_code', 'publication_date', 'publisher','ratings_count','ratings_count_per_num_pages','text_reviews_count'])

As we now expect all the values in the remaining dataframe to be purely numerical, we run a script to filter out all rows with non-numerical values

In [210]:
df = df.apply(pd.to_numeric, errors='coerce').dropna()

In [211]:
X = df.iloc[:, 1:df.shape[1]]
y = df['average_rating']

In [212]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [213]:
linmod1 = sm.OLS(y_train, sm.add_constant(X_train))
res_linmod1 = linmod1.fit()

In [173]:
y_pred = np.clip(res_linmod1.predict(X_test), 0, 5)
r_sq1 = r2_sore(y_test, y_pred)
print(f"R_squared = {r_sq1}")

ValueError: ignored

In [214]:
r_sq1 = res_linmod1.rsquared
print(f"R_squared = {r_sq1}")

R_squared = 0.09257069007689678


Before we look closer at ways of simplifying the linear regression model, we have a look at other regression models
We start with random forest.

In [291]:
rfmod1 = XGBRegressor(n_estimators=1000, max_depth=1, eta=0.2, colsample_bytree=0.9,random_state=1)
rfmod1.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, early_stopping_rounds=None,
             enable_categorical=False, eta=0.2, eval_metric=None,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=1,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

In [292]:
y_pred = np.clip(rfmod1.predict(X_test), 0, 5)
r_sq1 = r2_score(y_test, y_pred)
print(f"R_squared = {r_sq1}")

R_squared = 0.1119170082651324


In [293]:
r_sq2 = rfmod1.score(X_test, y_test)
print(f"R_squared = {r_sq2}")

R_squared = 0.1119170082651324


In [294]:
y_pred = rfmod1.predict(X_test)
largest_value = y_pred.max()
print(largest_value)
smallest_value = y_pred.min()
print(smallest_value)

4.435283
3.5363002
